# LAB Geo 1

In [32]:
from pymongo import MongoClient
from pprint import pprint

client = MongoClient()
db = client.geo1

## Exercise 1

In [9]:
db.states.create_index([('loc', '2dsphere')])
db.states.index_information()
db.states.drop_index([('loc', '2dsphere')])
db.states.index_information()

'loc_2dsphere'

## Exercise 2

In [28]:
given_zip = '10044'
state_code = db.small_zips.find_one({'_id': given_zip})['state']
state_area = db.states.find_one({'code': state_code})['loc']
res = db.airports.find(
    {'loc': {'$geoWithin': {'$geometry': state_area}}, 'type': 'International'}, 
    {'name': 1, '_id': 0}).sort([('name', 1)])
list(res)

[{'name': 'Albany County'},
 {'name': 'Greater Buffalo Intl'},
 {'name': 'Greater Rochester International'},
 {'name': 'John F Kennedy Intl'},
 {'name': 'La Guardia'},
 {'name': 'Massena Intl'},
 {'name': 'Niagara Falls Intl'},
 {'name': 'Ogdensburg Intl'},
 {'name': 'Stewart Intl'},
 {'name': 'Syracuse Hancock Intl'},
 {'name': 'Watertown International'}]

## Exercise 3

In [35]:
airports = db.airports.find({'type': 'International'})
for airport in airports:
    res = db.restaurants.find({
        'address.coord': {'$geoWithin': {'$centerSphere': [airport['loc']['coordinates'], 2 / 6378.1]}},
        'cuisine': 'Korean'})
    lres = list(res)
    if len(lres) == 0:
        continue
    pprint(lres)

[{'_id': ObjectId('660ad116de24ddeaf4341079'),
  'address': {'building': '0',
              'coord': [-73.7822056, 40.6434612],
              'street': 'Jfk International Airpor',
              'zipcode': '11430'},
  'borough': 'Queens',
  'cuisine': 'Korean',
  'grades': [{'date': datetime.datetime(2014, 6, 18, 0, 0),
              'grade': 'A',
              'score': 2},
             {'date': datetime.datetime(2014, 1, 14, 0, 0),
              'grade': 'A',
              'score': 12},
             {'date': datetime.datetime(2013, 1, 4, 0, 0),
              'grade': 'A',
              'score': 11},
             {'date': datetime.datetime(2012, 4, 25, 0, 0),
              'grade': 'A',
              'score': 13}],
  'name': 'Korean Lounge',
  'restaurant_id': '40625198'}]


In [38]:
db.restaurants.create_index([('address.coord', '2dsphere')])
airports = db.airports.find({'type': 'International'})
for airport in airports:
    res = db.restaurants.find({
        'address.coord': {'$nearSphere': {'$geometry': airport['loc'], '$maxDistance': 2000}},
        'cuisine': 'Korean'})
    lres = list(res)
    if len(lres) == 0:
        continue
    pprint(lres)

[{'_id': ObjectId('660ad116de24ddeaf4341079'),
  'address': {'building': '0',
              'coord': [-73.7822056, 40.6434612],
              'street': 'Jfk International Airpor',
              'zipcode': '11430'},
  'borough': 'Queens',
  'cuisine': 'Korean',
  'grades': [{'date': datetime.datetime(2014, 6, 18, 0, 0),
              'grade': 'A',
              'score': 2},
             {'date': datetime.datetime(2014, 1, 14, 0, 0),
              'grade': 'A',
              'score': 12},
             {'date': datetime.datetime(2013, 1, 4, 0, 0),
              'grade': 'A',
              'score': 11},
             {'date': datetime.datetime(2012, 4, 25, 0, 0),
              'grade': 'A',
              'score': 13}],
  'name': 'Korean Lounge',
  'restaurant_id': '40625198'}]


## Exercise 4

In [43]:
lax = db.airports.find_one({'code': 'LAX'})
dtw = db.airports.find_one({'code': 'DTW'})
geo_obj = {
    'type': 'LineString',
    'coordinates': [lax['loc']['coordinates'], dtw['loc']['coordinates']]
}

res = db.states.find({'loc': {'$geoIntersects': {'$geometry': geo_obj}}}, {'name': 1, '_id': 0}).sort([('name', 1)])
list(res)

[{'name': 'Arizona'},
 {'name': 'California'},
 {'name': 'Colorado'},
 {'name': 'Illinois'},
 {'name': 'Iowa'},
 {'name': 'Kansas'},
 {'name': 'Michigan'},
 {'name': 'Nebraska'},
 {'name': 'Nevada'},
 {'name': 'Utah'}]